# PROCESO DE EXTRACCIÓN - TRANSFORMACIÓN - LIMPIEZA DE LOS DATOS DE LA PLATAFORMA MULTINACIONAL "STREAM"

# IMPORTACIÓN DE LAS LIBRERIAS PARA EL PROCESO...

In [1]:
import os
import requests
from openpyxl import load_workbook
import ast
import pandas as pd
import numpy as np
import json
from textblob import TextBlob

# CREACIÓN DE LOS DATAFRAMES CORRESPONDIENTES A CADA ARCHIVO...

In [2]:
# Rutas a los archivos JSON:
rutas_archivos = ['DATASET_STEAM/australian_user_reviews.json',
                  'DATASET_STEAM/australian_users_items.json',
                  'DATASET_STEAM/output_steam_games.json']

# Crear listas para almacenar los datos de cada archivo
australian_user_reviews = []
australian_users_items = []
output_steam_games = []

# Procesar cada archivo por separado
for ruta_archivo in rutas_archivos:
    if 'australian_user_reviews' in ruta_archivo:
        data_list = australian_user_reviews
    elif 'australian_users_items' in ruta_archivo:
        data_list = australian_users_items
    elif 'output_steam_games' in ruta_archivo:
        data_list = output_steam_games

    with open(ruta_archivo, encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line)
            except json.JSONDecodeError:
                data = ast.literal_eval(line)
            data_list.append(data)

# Crear DataFrames para cada conjunto de datos
df_reviews = pd.DataFrame(australian_user_reviews)
df_items = pd.DataFrame(australian_users_items)
df_games = pd.DataFrame(output_steam_games)

# PROCESO DE TRATAMIENTO DE CADA DATAFRAME CORESPONDIENTE A CADA ARCHIVO DEL DATASET...

## DATAFRAME "australian_user_reviews"

In [3]:
# EXPLORACIÓN INICIAL DE DATOS...

df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [4]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [5]:
# EXTRACIÓN DE LOS DATOS DE LAS LISTAS Y DICCIONARIOS...

# Extraer el diccionario de la lista
df_reviews['reviews'] = df_reviews['reviews'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else {})

# Crear un nuevo DataFrame con los valores del diccionario
df_reviews_reviews = pd.json_normalize(df_reviews['reviews'])

# eliminar columnas
df_reviews = df_reviews.drop(['reviews'], axis=1)

# Concatenar
df_reviews = pd.concat([df_reviews, df_reviews_reviews], axis=1)

In [6]:
df_reviews.head(1)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...


In [7]:
# para saber si hay columnas irrelevantes ( que no aportan nada al análisis) columna irrelevante = 1
analisis_subniveles = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']

for col in analisis_subniveles:
    print(f'Columna {col}: {df_reviews[col].nunique()} subniveles')

Columna user_id: 25485 subniveles
Columna user_url: 25485 subniveles
Columna funny: 170 subniveles
Columna posted: 1549 subniveles
Columna last_edited: 855 subniveles
Columna item_id: 2459 subniveles
Columna helpful: 991 subniveles
Columna recommend: 2 subniveles
Columna review: 24035 subniveles


In [8]:
# EXPLORACIÓN DE DATOS...

df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25799 non-null  object
 1   user_url     25799 non-null  object
 2   funny        25771 non-null  object
 3   posted       25771 non-null  object
 4   last_edited  25771 non-null  object
 5   item_id      25771 non-null  object
 6   helpful      25771 non-null  object
 7   recommend    25771 non-null  object
 8   review       25771 non-null  object
dtypes: object(9)
memory usage: 1.8+ MB


In [9]:
# ELIMINACIÓN DE DATOS REPETIDOS...

# Para los valores repetidos
print(f'Tamaño del Dataset antes de eliminar filas repetidas: {df_reviews.shape}')
df_reviews.drop_duplicates(inplace=True)
print(f'Tamaño del Dataset después de eliminar filas repetidas: {df_reviews.shape}')

Tamaño del Dataset antes de eliminar filas repetidas: (25799, 9)
Tamaño del Dataset después de eliminar filas repetidas: (25486, 9)


In [10]:
# REEMPLAZO DE VALORES FALTANTES EN EL DATAFRAME...

# Relleno de valores faltantes
df_reviews = df_reviews.fillna(np.nan)

In [11]:
# CREACIÓN DE LA COLUMNA SENTIMENT_ANALYSIS PARA EL ANÁLISIS DE SENTIMIENTOS CON PNL...

# Crear una función para realizar el análisis de sentimiento
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # Clasificar el sentimiento en función de la polaridad
    if analysis.sentiment.polarity < 0:
        return 0  # Negativo
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
    else:
        return 2  # Positivo

# Filtrar solo los valores que no son NaN
df_reviews = df_reviews.dropna(subset=['review'])

# Aplicar el análisis de sentimiento y almacenar los resultados en una nueva columna
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

# Reemplazar los valores en la escala requerida (0 para malo, 1 para neutral y 2 para positivo)
df_reviews['sentiment_analysis'].replace({0: 0, 1: 1, 2: 2}, inplace=True)

In [12]:
df_reviews.head(1)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2


In [13]:
# CARGA DE LOS DATOS EN UN ARCHIVO CON FORMATO CSV

# Crear una carpeta llamada "DATASET_STEAM_LIMPIO" si no existe
carpeta_destino = "DATASET_STEAM_LIMPIO"
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Guardar el DataFrame en formato csv en la carpeta
ruta_destino = os.path.join(carpeta_destino, "reviews_limpio.csv")
df_reviews.to_csv(ruta_destino, index=False)

print("DataFrame guardado en formato csv")

DataFrame guardado en formato csv


## DATAFRAME "australian_users_items"

In [14]:
# EXPLORACIÓN INICIAL DE DATOS...

df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [15]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [16]:
# EXTRACIÓN DE LOS DATOS DE LAS LISTAS Y DICCIONARIOS...

# Extraer el diccionario de la lista
df_items['items'] = df_items['items'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else {})

# Crear un nuevo DataFrame con los valores del diccionario
df_items_items = pd.json_normalize(df_items['items'])

# eliminar columnas
df_items = df_items.drop(['items'], axis=1)

# Concatenar
df_items = pd.concat([df_items, df_items_items], axis=1)

In [17]:
df_items.head(1)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0


In [18]:
# ESTUDIO DE CADA COLUMNA CON SUS DATOS...

# para saber si hay columnas irrelevantes ( que no aportan nada al análisis) columna irrelevante = 1
analisis_subniveles = ['user_id', 'items_count', 'steam_id', 'user_url', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']

for col in analisis_subniveles:
    print(f'Columna {col}: {df_items[col].nunique()} subniveles')

Columna user_id: 87626 subniveles
Columna items_count: 925 subniveles
Columna steam_id: 87625 subniveles
Columna user_url: 87626 subniveles
Columna item_id: 1199 subniveles
Columna item_name: 1195 subniveles
Columna playtime_forever: 12836 subniveles
Columna playtime_2weeks: 845 subniveles


In [19]:
# ELIMINACIÓN DE DATOS REPETIDOS...

# para los valores repetidos
print(f'Tamaño del Dataset antes de eliminar filas repetidas: {df_items.shape}')
df_items.drop_duplicates(inplace=True)
print(f'Tamaño del Dataset después de eliminar filas repetidas: {df_items.shape}')

Tamaño del Dataset antes de eliminar filas repetidas: (88310, 8)
Tamaño del Dataset después de eliminar filas repetidas: (87632, 8)


In [20]:
# REEMPLAZO DE VALORES FALTANTES EN EL DATAFRAME...

# rellenar los valores faltantes por medio de np.nan
df_items = df_items.fillna(np.nan)

In [21]:
# CARGA DE LOS DATOS EN UN ARCHIVO CON FORMATO CSV...

# Crear una carpeta llamada "DATASET_STEAM_LIMPIO" si no existe
carpeta_destino = "DATASET_STEAM_LIMPIO"
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Guardar el DataFrame en formato csv en la carpeta
ruta_destino = os.path.join(carpeta_destino, "items_limpio.csv")
df_items.to_csv(ruta_destino, index=False)

print("DataFrame guardado en formato csv")

DataFrame guardado en formato csv


## DATAFRAME "output_steam_games"

In [22]:
# EXPLORACIÓN INICIAL DE DATOS...

df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [23]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [24]:
# ELIMINACIÓN DE FILAS INNECESARIAS PARA EL PROYECTO...

# Eliminar filas que contienen valores vacíos en todas las columnas
df_games = df_games.dropna(how='all', axis=0)

In [25]:
# TRATAMIENTO DE LA COLUMNA "genre" PARA QUE CADA GENERO TENGA SU PROPIA FILA...

# Utilizar 'explode' para crear una fila por cada género
df_games = df_games.explode('genres', ignore_index=True)

# Duplicar los valores en las demás columnas
df_games = df_games.join(df_games['genres'].apply(pd.Series).rename(lambda x: f'genre_{x}', axis=1))

In [26]:
# TRATAMIENTO DE LAS COLUMNAS QUE POSEEN LISTAS EN EL DATAFRAME...

# Asegurarse de que todos los valores sean listas
df_games['tags'] = df_games['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_games['specs'] = df_games['specs'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [27]:
# ELIMINACIÓN DE DATOS REPETIDOS...

# para los valores repetidos
print(f'Tamaño del Dataset antes de eliminar filas repetidas: {df_games.shape}')
df_games.drop_duplicates(inplace=True)
print(f'Tamaño del Dataset después de eliminar filas repetidas: {df_games.shape}')

Tamaño del Dataset antes de eliminar filas repetidas: (74837, 14)
Tamaño del Dataset después de eliminar filas repetidas: (74837, 14)


In [28]:
# ESTUDIO DE CADA COLUMNA CON SUS DATOS...

# para saber si hay columnas irrelevantes ( que no aportan nada al análisis) columna irrelevante = 1
analisis_subniveles = ['publisher', 'genres', 'app_name', 'title', 'url', 'release_date', 'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id', 'developer']
for col in analisis_subniveles:
    print(f'Columna {col}: {df_games[col].nunique()} subniveles')

Columna publisher: 8239 subniveles
Columna genres: 22 subniveles
Columna app_name: 32094 subniveles
Columna title: 30054 subniveles
Columna url: 32135 subniveles
Columna release_date: 3582 subniveles
Columna tags: 15395 subniveles
Columna reviews_url: 32132 subniveles
Columna specs: 4649 subniveles
Columna price: 162 subniveles
Columna early_access: 2 subniveles
Columna id: 32132 subniveles
Columna developer: 10992 subniveles


In [29]:
# REEMPLAZO DE VALORES FALTANTES EN EL DATAFRAME...

# rellenar los valores faltantes por medio de np.nan
df_games = df_games.fillna(np.nan)

In [30]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,genre_0
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"Strategy, Action, Indie, Casual, Simulation",http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140,Kotoshiro,Action


In [31]:
# CARGA DE LOS DATOS EN UN ARCHIVO CON FORMATO CSV...

# Crear una carpeta llamada "DATASET_STEAM_LIMPIO" si no existe
carpeta_destino = "DATASET_STEAM_LIMPIO"
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Guardar el DataFrame en formato csv en la carpeta
ruta_destino = os.path.join(carpeta_destino, "games_limpio.csv")
df_games.to_csv(ruta_destino, index=False)

print("DataFrame guardado en formato csv")

DataFrame guardado en formato csv
